<a href="https://colab.research.google.com/github/divyam1464/Text-Summarizer/blob/main/TextSummarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers accelerate datasets

In [1]:
from datasets import load_dataset
multinews = load_dataset("multi_news", split="test")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

multi_news.py: 0.00B [00:00, ?B/s]

RuntimeError: Dataset scripts are no longer supported, but found multi_news.py

In [6]:
multinews.to_pandas()

NameError: name 'multinews' is not defined

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
multinews = multinews.train_test_split(test_size = 0.2)

In [ ]:
prefix = 'summarize'
def process_function(examples):
  inputs = [prefix + doc for doc in examples['document']]
  model_input = tokenizer(inputs, max_length = 1024, truncation = True)
  labels = tokenizer(text = examples['summary'], max_length = 128, truncation = True)
  model_input['labels'] = labels['input_ids']
  return model_input

In [ ]:
tokenized_multinews = multinews.map(process_function, batched = True)

In [ ]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = 't5-small')
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

In [ ]:
trainings_args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    eval_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 10,
    per_device_eval_batch_size = 10,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 10,
    fp16 = True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model = model,
    args = trainings_args,
    train_dataset = tokenized_multinews['train'],
    eval_dataset = tokenized_multinews['test'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    )

In [ ]:
trainer.train()

In [ ]:
text = """
Feature extraction is the process of transforming raw data into a simplified and informative set of features or attributes. This reduces data complexity and highlights the most relevant information making it easier for machine learning models to analyze and learn from the data efficiently. It plays an important role in improving model accuracy and reducing computational costs by focusing on essential aspects of the data.

key_techniques_for_feature_extraction.webpkey_techniques_for_feature_extraction.webp
Importance of Feature Extraction
Feature extraction is important for several reasons:

Reduced Computation Cost: Raw data, especially from images or large datasets can be very complex. Feature extraction makes this data simpler hence reducing the computational resources needed for processing.
Improved Model Performance: By focusing on key features, machine learning models can work with more relevant information leading to better performance and more accurate results.
Better Insights: Reducing the number of features helps algorithms concentrate on the most important data, eliminating noise and irrelevant information which can lead to deeper insights.
Prevention of Overfitting: Models with too many features may become too specific to the training data, making them perform poorly on new data. Feature extraction reduces this risk by simplifying the model.
Key Techniques for Feature Extraction
There are various techniques for extracting meaningful features from different types of data:

1. Statistical Methods
Statistical methods are used in feature extraction to summarize and explain patterns of data. Common data attributes include:

stat
Statistical Methods
Mean: The average value of a dataset.
Median: The middle value when it is sorted in ascending order.
Standard Deviation: A measure of the spread or dispersion of a sample.
Correlation and Covariance: Measures of the linear relationship between two or more factors.
Regression Analysis: A way to model the link between a dependent variable and one or more independent factors.
These statistical methods can be used to represent the center trend, spread and links within a collection.

2. Dimensionality Reduction
Dimensionality reduction reduces the number of features without losing important information. Some popular methods are:

dim2.pngdim2.png
Principal Component Analysis: It selects variables that account for most of the data’s variation, simplifying the dataset by focusing on the most important components.
Linear Discriminant Analysis (LDA): It finds the best combination of features to separate different classes, maximizing class separability for better classification.
t-Distributed Stochastic Neighbor Embedding (t-SNE): A technique that reduces high-dimensional data into two or three dimensions ideal for visualizing complex datasets.
3. Feature Extraction for Textual Data
In Natural Language Processing (NLP), we often convert raw text into a format that machine learning models can understand. Some common techniques are:

Bag of Words (BoW): Represents a document by counting word frequencies, ignoring word order, useful for basic text classification.
Term Frequency-Inverse Document Frequency (TF-IDF): Adjusts word importance based on frequency in a specific document compared to all documents, highlighting unique terms.
4. Signal Processing Methods
It is used for analyzing time-series, audio and sensor data:

origsig
Signal processing methods
Fourier Transform:It converts a signal from the time domain to the frequency domain to analyze its frequency components.
Wavelet Transform:It analyzes signals that vary over time, offering both time and frequency information for non-stationary signals.
5. Image Data Extraction
Techniques for extracting features from images:

cnnhog
Image Data Extraction
Histogram of Oriented Gradients (HOG):This technique finds the distribution of intensity gradients or edge directions in an image. It's used in object detection and recognition tasks.
Convolutional Neural Networks (CNN) Features: They learn hierarchical features from images through layers of convolutions, ideal for classification and detection tasks.
Choosing the Right Method
Selecting the appropriate feature extraction method depends on the type of data and the specific problem we're solving. It requires careful consideration and often domain expertise.

Information Loss: Feature extraction might simplify the data too much, potentially losing important information in the process.
Computational Complexity: Some methods, especially for large datasets can be computationally expensive and may require significant resources.
"""

In [ ]:
input_ids = tokenizer(text, max_length = 1024, truncation = True, return_tensors = 'pt').input_ids
input_ids = input_ids.to('cuda')

In [ ]:
import torch
with torch.no_grad():
  if model.device.type == 'cuda':
    output = model.generate(input_ids, max_length = 150, num_beams = 5)

summary_ids = output[0].tolist()
summary = tokenizer.decode(summary_ids, skip_special_tokens = True)
print(summary)


In [ ]:
summary

In [ ]:
ref_summary = """Feature extraction transforms raw data into a simplified set of attributes, enhancing analysis efficiency for machine learning models. It reduces data complexity, leading to lower computational costs, improved model performance, more profound insights, and prevention of overfitting. Key techniques include statistical methods for summarizing data attributes, dimensionality reduction approaches like Principal Component Analysis and t-SNE, textual data extraction methods such as Bag of Words and TF-IDF, signal processing techniques like Fourier and Wavelet transforms, and image data extraction methods including Histogram of Oriented Gradients and Convolutional Neural Networks. Choosing the right feature extraction method is crucial and depends on the data type and specific problem."""

In [ ]:
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(summary, ref_summary)
scores

In [ ]:
trainer.save_model()

In [ ]:
model.save_pretrained('Summarizer')